# Joint 3D analysis
This tutorial shows how to run a joint 3D map-based analysis using three example observations of the Galactic center region with CTA.

In [ ]:
## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion, make_psf
from gammapy.maps import WcsGeom, MapAxis, Map
from gammapy.cube import MapMaker, PSFKernel, MapDataset
from gammapy.cube.models import SkyModel, BackgroundModel
from gammapy.spectrum.models import PowerLaw
from gammapy.image.models import SkyPointSource
from gammapy.utils.fitting import Fit

## Prepare modeling input data

### Prepare input maps

We first use the `DataStore` object to access the CTA observations and retrieve a list of observations by passing the observations IDs to the `.get_observations()` method:

In [ ]:
# Define which data to use and print some information
data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")

In [ ]:
# Select some observations from these dataset by hand
obs_ids = [110380, 111140, 111159]
observations = data_store.get_observations(obs_ids)

Now we define a reference geometry for our analysis, We choose a WCS based gemoetry with a binsize of 0.02 deg and also define an energy axis: 

In [ ]:
# Source position
src_pos = SkyCoord(0, 0, unit="deg", frame="galactic")

energy_axis = MapAxis.from_edges(
    np.logspace(-1.0, 1.0, 10), unit="TeV", name="energy", interp="log"
)

# Here are defined the common parameters for the geometry of the different observation
geom_kwargs = {
    "binsz": 0.02,
    "width": (8, 8),
    "coordsys": "GAL",
    "proj": "CAR",
    "axes": [energy_axis],
}

offset_max = 4.0 * u.deg

The `MapMaker` object is initialized with this reference geometry and a field of view cut of 4 deg.

The maps are prepared by calling the `.run()` method and passing the `observations`. The `.run()` method returns a Python `dict` containing a `counts`, `background` and `exposure` map. For the joint analysis, we compute the cube per observation.

In [ ]:
%%time
maps_list = {}

for obs in observations:
    # For each observation, the map will be centered on the pointing position.
    geom = WcsGeom.create(skydir=obs.pointing_radec, **geom_kwargs)
    maker = MapMaker(geom, offset_max=offset_max)
    maps = maker.run([obs])
    maps_list[obs.obs_id] = maps

### Prepare IRFs
PSF and Edisp are estimated for each observation at a specific source position src_pos
  

In [ ]:
# define energy grid for edisp
energy = energy_axis.edges * energy_axis.unit

In [ ]:
irf_list = {}

for obs in observations:
    irf_list[obs.obs_id] = {}
    table_psf = make_psf(obs, src_pos)
    irf_list[obs.obs_id]["psf"] = PSFKernel.from_table_psf(
        table_psf, geom, max_radius="0.3 deg"
    )

    offset = src_pos.separation(obs.pointing_radec)
    irf_list[obs.obs_id]["edisp"] = obs.edisp.to_energy_dispersion(
        offset, e_true=energy, e_reco=energy
    )

Save maps as well as IRFs to disk:

In [ ]:
for obs_id in obs_ids:
    path = Path("analysis_3d_joint") / "obs_{}".format(obs_id)
    path.mkdir(parents=True, exist_ok=True)

    for key in ["counts", "exposure", "background"]:
        filename = "{}.fits.gz".format(key)
        maps_list[obs_id][key].write(path / filename, overwrite=True)

    for key in ["psf", "edisp"]:
        filename = "{}.fits.gz".format(key)
        irf_list[obs_id][key].write(path / filename, overwrite=True)

## Likelihood fit

### Reading maps and IRFs
As first step we read in the maps and IRFs for each observations

In [ ]:
maps_datasets = {}
for obs_id in obs_ids:
    path = Path("analysis_3d_joint") / "obs_{}".format(obs.obs_id)
    maps_dataset = {}

    for keys in ["counts", "exposure", "background"]:
        maps_dataset[keys] = Map.read(path / "{}.fits.gz".format(keys))

    maps_dataset["psf"] = PSFKernel.read(path / "psf.fits.gz")
    maps_dataset["edisp"] = EnergyDispersion.read(path / "edisp.fits.gz")
    maps_datasets[obs_id] = maps_dataset

Define source model:

In [ ]:
spatial_model = SkyPointSource(lon_0="0.01 deg", lat_0="0.01 deg")
spectral_model = PowerLaw(
    index=2.2, amplitude="3e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

Create the dataset for each observation:

In [ ]:
datasets = []

# Here we defined energy threshold per observation
energy_thresholds = {
    110380: 0.2 * u.TeV,
    111140: 0.5 * u.TeV,
    111159: 0.3 * u.TeV,
}

for obs_id in obs_ids:
    map_dataset = maps_datasets[obs_id]

    # create background model per observation / dataset
    background_model = BackgroundModel(maps_dataset["background"])
    # TODO: move this to WcsGeom.energy_mask()?
    counts = maps_dataset["counts"]
    mask = Map.from_geom(counts.geom)
    coords = mask.geom.get_coord()
    mask.data = coords["energy"] > energy_thresholds[obs_id].to_value("TeV")

    dataset = MapDataset(
        model=model,
        counts=counts,
        exposure=maps_dataset["exposure"],
        psf=maps_dataset["psf"],
        edisp=maps_dataset["edisp"],
        background_model=background_model,
        mask=mask,
    )
    datasets.append(dataset)

In [ ]:
fit = Fit(datasets)

In [ ]:
%%time
result = fit.run()

In [ ]:
print(result)

Best fit parameters:

In [ ]:
fit.datasets.parameters.to_table()

## Plotting residuals

In [ ]:
def plot_residuals(dataset):
    npred = dataset.npred()
    residual = (dataset.counts - npred).smooth("0.1 deg")

    residual.plot_interactive(
        vmin=-0.1, vmax=0.1, cmap="coolwarm", add_cbar=True, stretch="linear"
    )

Each observation has different energy threshold. Keep in mind that the residuals are not meaningful below the energy threshold.

In [ ]:
plot_residuals(datasets[0])

In [ ]:
plot_residuals(datasets[1])

In [ ]:
plot_residuals(datasets[2])